"""

    Collaborative-based filtering for item recommendation.

    Author: Explore Data Science Academy.

    Note:
    ---------------------------------------------------------------------
    Please follow the instructions provided within the README.md file
    located within the root of this repository for guidance on how to use
    this script correctly.

    NB: You are required to extend this baseline algorithm to enable more
    efficient and accurate computation of recommendations.

    !! You must not change the name and signature (arguments) of the
    prediction function, `collab_model` !!

    You must however change its contents (i.e. add your own collaborative
    filtering algorithm), as well as altering/adding any other functions
    as part of your improvement.

    ---------------------------------------------------------------------

    Description: Provided within this file is a baseline collaborative
    filtering algorithm for rating predictions on Movie data.

"""

In [1]:
# Script dependencies
import pandas as pd
import numpy as np
import scipy as sp
import pickle
import copy
from surprise import Reader, Dataset
from surprise import SVD, NormalPredictor, BaselineOnly, KNNBasic, NMF
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
# Importing data
movies_df = pd.read_csv('../data/movies.csv',sep = ',',delimiter=',')
ratings_df = pd.read_csv('../data/ratings.csv')
ratings_df.drop(['timestamp'], axis=1,inplace=True)

In [3]:
# We make use of an SVD model trained on a subset of the MovieLens 10k dataset.
model=pickle.load(open('../models/SVD.pkl', 'rb'))

In [4]:
def prediction_item(item_id):
    """Map a given favourite movie to users within the
       MovieLens dataset with the same preference.

    Parameters
    ----------
    item_id : int
        A MovieLens Movie ID.

    Returns
    -------
    list
        User IDs of users with similar high ratings for the given movie.

    """
    # Data preprosessing
    reader = Reader(rating_scale=(0, 5))
    load_df = Dataset.load_from_df(ratings_df,reader)
    a_train = load_df.build_full_trainset()

    predictions = []
    for ui in a_train.all_users():
        predictions.append(model.predict(iid=item_id,uid=ui, verbose = False))
    return predictions

In [5]:
def pred_movies(movie_list):
    """Maps the given favourite movies selected within the app to corresponding
    users within the MovieLens dataset.

    Parameters
    ----------
    movie_list : list
        Three favourite movies selected by the app user.

    Returns
    -------
    list
        User-ID's of users with similar high ratings for each movie.

    """
    # Store the id of users
    id_store=[]
    # For each movie selected by a user of the app,
    # predict a corresponding user within the dataset with the highest rating
    for i in movie_list:
        predictions = prediction_item(item_id = i)
        predictions.sort(key=lambda x: x.est, reverse=True)
        # Take the top 10 user id's from each movie with highest rankings
        for pred in predictions[:10]:
            id_store.append(pred.uid)
    # Return a list of user id's
    return id_store

In [16]:
# !! DO NOT CHANGE THIS FUNCTION SIGNATURE !!
# You are, however, encouraged to change its content.  
def collab_model(movie_list,top_n=10):
    """Performs Collaborative filtering based upon a list of movies supplied
       by the app user.

    Parameters
    ----------
    movie_list : list (str)
        Favorite movies chosen by the app user.
    top_n : type
        Number of top recommendations to return to the user.

    Returns
    -------
    list (str)
        Titles of the top-n movie recommendations to the user.

    """
    indices = pd.Series(movies_df['title'])
    user_ids = pred_movies(movie_list)
    df_init_users = ratings_df[ratings_df['userId']==user_ids[0]]
    for i in user_ids:
        df_init_users=df_init_users.append(ratings_df[ratings_df['userId']==i])

    # get util matrix
    util_matrix = df_init_users.pivot_table(index='userId',
                                            columns='movieId',
                                            values='rating')
#     return df_init_users
    # normalize
    util_matrix_norm = util_matrix.apply(lambda x: (x-np.mean(x))/(np.max(x)-np.min(x)), axis=1)
    # fillna, transpose, drop
    util_matrix_norm.fillna(0,inplace=True)
    util_matrix_norm=util_matrix_norm.T
    util_matrix_norm = util_matrix_norm.loc[:, (util_matrix_norm!=0).any(axis=0)]
    #save to sparse
    util_matrix_sparse = sp.sparse.csr_matrix(util_matrix_norm.values)
#     return indices
    # Getting the cosine similarity matrix
    cosine_sim = cosine_similarity(util_matrix_sparse.T)
    return pd.DataFrame(cosine_sim)
    idx_1 = indices[indices == movie_list[0]].index[0]
    idx_2 = indices[indices == movie_list[1]].index[0]
    idx_3 = indices[indices == movie_list[2]].index[0]
    # Creating a Series with the similarity scores in descending order
    rank_1 = cosine_sim[idx_1]
    rank_2 = cosine_sim[idx_2]
    rank_3 = cosine_sim[idx_3]
    # Calculating the scores
    score_series_1 = pd.Series(rank_1).sort_values(ascending = False)
    score_series_2 = pd.Series(rank_2).sort_values(ascending = False)
    score_series_3 = pd.Series(rank_3).sort_values(ascending = False)
     # Appending the names of movies
    listings = score_series_1.append(score_series_1).append(score_series_3).sort_values(ascending = False)
    recommended_movies = []
    # Choose top 50
    top_50_indexes = list(listings.iloc[1:50].index)
    # Removing chosen movies
    top_indexes = np.setdiff1d(top_50_indexes,[idx_1,idx_2,idx_3])
    for i in top_indexes[:top_n]:
        recommended_movies.append(list(movies_df['title'])[i])
    return recommended_movies

In [17]:
collab_model(['Sabotage (2014)','GoldenEye (1995)', 'Jumanji (1995)'],10)

,0,1,2,3,4,5,6,7,8,9
0,1.000000,-0.007015,0.000000,0.033453,-0.039406,-0.002471,0.000000,0.000000,0.000000,0.002312
1,-0.007015,1.000000,-0.008455,0.008584,0.005102,0.004127,0.000000,-0.008783,-0.045356,0.000000
2,0.000000,-0.008455,1.000000,-0.001073,0.066496,0.000000,0.000000,0.014446,0.013665,-0.006758
3,0.033453,0.008584,-0.001073,1.000000,-0.042605,-0.013009,0.006305,0.011085,0.035140,0.077326
4,-0.039406,0.005102,0.066496,-0.042605,1.000000,-0.029413,0.046042,0.049066,-0.018193,0.000000
5,-0.002471,0.004127,0.000000,-0.013009,-0.029413,1.000000,-0.017724,-0.007985,0.040465,-0.019364
6,0.000000,0.000000,0.000000,0.006305,0.046042,-0.017724,1.000000,-0.025342,-0.012529,0.003949
7,0.000000,-0.008783,0.014446,0.011085,0.049066,-0.007985,-0.025342,1.000000,0.000000,0.006671
8,0.000000,-0.045356,0.013665,0.035140,-0.018193,0.040465,-0.012529,0.000000,1.000000,0.006761
9,0.002312,0.000000,-0.006758,0.077326,0.000000,-0.019364,0.003949,0.006671,0.006761,1.000000


In [ ]:
mo

In [49]:
b

NameError: name 'b' is not defined